In [1]:
#Example NPZ FILE
!mkdir -p data/PittsburghBridge
!wget -P data/PittsburghBridge https://dl.fbaipublicfiles.com/pytorch3d/data/PittsburghBridge/pointcloud.npz

--2025-01-09 20:43:41--  https://dl.fbaipublicfiles.com/pytorch3d/data/PittsburghBridge/pointcloud.npz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.164.78.121, 18.164.78.81, 18.164.78.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.164.78.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5701352 (5.4M) [application/zip]
Saving to: ‘data/PittsburghBridge/pointcloud.npz’

pointcloud.npz      100%[===================>]   5.44M  12.1MB/s    in 0.4s    

2025-01-09 20:43:42 (12.1 MB/s) - ‘data/PittsburghBridge/pointcloud.npz’ saved [5701352/5701352]



In [2]:
import sys
import torch

need_pytorch3d = False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d = True
if need_pytorch3d:
    pyt_version_str = torch.__version__.split("+")[0].replace(".", "")
    version_str = "".join([
        f"py3{sys.version_info.minor}_cu",
        torch.version.cuda.replace(".", ""),
        f"_pyt{pyt_version_str}"
    ])
    !pip install iopath
    if sys.platform.startswith("linux"):
        print("Trying to install wheel for PyTorch3D")
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
        pip_list = !pip freeze
        need_pytorch3d = not any(i.startswith("pytorch3d==") for i in pip_list)
    if need_pytorch3d:
        print(f"failed to find/install wheel for {version_str}")
if need_pytorch3d:
    print("Installing PyTorch3D from source")
    !pip install ninja
    !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=60009e8417d5188a75d106de3ca3f5e4e10b673af3dcd44a3ac6e3f2fe706c71
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath
Trying to install wheel for PyTorch3D
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt241/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 66.9 MB/s eta 0:00:00a 0:00:01


In [3]:
from pytorch3d.structures import Pointclouds

def bounding_sphere_normalize(points: torch.Tensor) -> torch.Tensor:
    """
    points: (N,3) tensor of point coords
    Return normalized points in a unit sphere centered at origin.
    """
    center = points.mean(dim=0, keepdim=True)
    max_dist = (points - center).norm(p=2, dim=1).max()
    points_normed = (points - center) / max_dist
    return points_normed


def load_3d_data(file_path, num_points=10000, device="cuda", do_normalize=True):
    # Load NPZ point cloud directly like in the example
    pointcloud = np.load(file_path)
    verts = torch.Tensor(pointcloud['verts']).to(device)
    rgb = torch.Tensor(pointcloud['rgb']).to(device)

    # Subsample if needed
    if len(verts) > num_points:
        idx = torch.randperm(len(verts))[:num_points]
        verts = verts[idx]
        rgb = rgb[idx]

    if do_normalize:
        verts = bounding_sphere_normalize(verts)

    # Return both the points tensor and the Pointclouds object
    point_cloud = Pointclouds(points=[verts], features=[rgb])
    return point_cloud  # Return both



In [4]:
from itertools import islice

import torch
from pytorch3d.structures import Pointclouds
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor
)
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torchvision.transforms as T


class MultiViewPointCloudRenderer:
    def __init__(self, image_size=512, base_dist=20, base_elev=10, base_azim=0,
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        self.device = device
        self.image_size = image_size
        self.base_dist = base_dist
        self.base_elev = base_elev
        self.base_azim = base_azim
        self.to_tensor = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor()
        ])

        # Define the settings for rasterization
        self.raster_settings = PointsRasterizationSettings(
            image_size=image_size,
            radius=0.003,
            points_per_pixel=10
        )

        # Define all views relative to base view
        self.views = {
            'Default': (base_dist, base_elev, base_azim),
            'Y_90deg': (base_dist, base_elev, base_azim + 90),
            'Y_180deg': (base_dist, base_elev, base_azim + 180),
            'Y_-90deg': (base_dist, base_elev, base_azim - 90),
            'X_90deg': (base_dist, base_elev + 90, base_azim),
            'X_-90deg': (base_dist, base_elev - 90, base_azim),
        }


    def get_center_point(self, point_cloud):
        """Calculate the center point of the point cloud"""
        points = point_cloud.points_packed()
        center = torch.mean(points, dim=0)
        return center.unsqueeze(0)  # Add batch dimension

    def create_renderer(self, dist, elev, azim, center_point, background_color=(0, 0, 0)):
        """Create a renderer for specific camera parameters"""
        # Use the center point as the 'at' parameter
        R, T = look_at_view_transform(
            dist=dist,
            elev=elev,
            azim=azim,
            at=center_point,  # Look at the center of the point cloud
        )
        cameras = FoVOrthographicCameras(device=self.device, R=R, T=T, znear=0.01)

        rasterizer = PointsRasterizer(cameras=cameras, raster_settings=self.raster_settings)
        renderer = PointsRenderer(
            rasterizer=rasterizer,
            compositor=AlphaCompositor(background_color=background_color)
        )
        return renderer

    def load_background(self, background_path):
        bg_image = Image.open(background_path)
        bg_tensor = self.to_tensor(bg_image).to(self.device)
        return bg_tensor.permute(1, 2, 0)  # Convert to HWC format

    def render_all_views(self, point_cloud, n_views=6, background_path=None,background_color=(0, 0, 0)):
        images = {}
        center_point = self.get_center_point(point_cloud)

        if background_path:
            background = self.load_background(background_path)
        else:
            background = None

        for view_name, (dist, elev, azim) in islice(self.views.items(), n_views):
            renderer = self.create_renderer(dist, elev, azim, center_point,background_color=background_color)
            image = renderer(point_cloud)

            if background is not None:
                # Create binary mask from points
                mask = torch.any(image[0, ..., :3] > 0, dim=-1).float()
                mask = mask.unsqueeze(-1).expand(-1, -1, 3)
                composite = (image[0, ..., :3] * mask) + (background * (1 - mask))
                images[view_name] = composite
            else:
                images[view_name] = image[0, ..., :3]

        return images

In [5]:
import os
import numpy as np
import torch
import torchvision

def save_results(point_cloud, renderer,n_views,device,output_dir,output_name):
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    rendered_images = renderer.render_all_views(point_cloud=point_cloud, n_views=n_views,background_color = (1,1,1))
    # Convert dictionary of images to tensor
    rendered_tensor = []
    for name, img in rendered_images.items():
        rendered_tensor.append(img.to(device))
    rendered_tensor = torch.stack(rendered_tensor)

    # Convert rendered images to CLIP format
    rendered_images = rendered_tensor.permute(0, 3, 1, 2)  # [B, H, W, C] -> [B, C, H, W]

    # Convert to uint8 range [0, 255]
    rendered_images = (rendered_images * 255).clamp(0, 255).to(torch.uint8)

    # Save rendered image using torchvision
    torchvision.utils.save_image(
        rendered_images.float() / 255.0,  # Convert back to [0,1] range
        os.path.join(output_dir, output_name),
         normalize=False  # We've already normalized the values
     )

In [6]:
device="cuda"

point_cloud = load_3d_data(
    "/kaggle/working/data/PittsburghBridge/pointcloud.npz",
    num_points=100000
)


renderer = MultiViewPointCloudRenderer(
    image_size=512,
    base_dist=30,  # Your default view distance
    base_elev=10,  # Your default elevation
    base_azim=45,  # Your default azimuth
    device=device
)

save_results(
    point_cloud=point_cloud,
    renderer=renderer,
    n_views=1,
    output_dir="./output",
    output_name="point_cloud.png",
    device=device
)